In [ ]:
import xarray as xr
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import xesmf as xe
from pathlib import Path
import itertools
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely.prepared import prep

Cargamos los modelos en un dataxarray

In [ ]:
def abrir_modelos(variable, ruta_base=None):
    """
    Abrir todos los archivos .nc en la subcarpeta correspondiente a la variable dentro de 'modelos'

    Args:  
      variable (str): nombre de la variable (subcarpeta dentro de 'modelos').
      ruta_base (str, opcional): ruta base donde se encuentra la carpeta 'modelos'.
                                 Si no se proporciona, se asume que está en el mismo
                                 directorio que este script.
    Returns:
      list[xarray.Dataset]: lista con los datasets abiertos.

    Raises:
      FileNotFoundError: si la carpeta no existe o no contiene archivos .nc.
    """
    try:
        base = Path(__file__).parent  # disponible si se ejecuta como script
    except NameError:
        base = Path.cwd()
    # Resolver ruta_base: por defecto 'modelos' relativa a este archivo,
    # pero permite pasar una ruta absoluta o relativa si se desea.
    if ruta_base:
        modelos_root = Path(ruta_base)
        if not modelos_root.is_absolute():
            modelos_root = base / modelos_root
    else:
        modelos_root = base / 'modelos'

    ruta_variable = modelos_root / variable
    if not ruta_variable.exists() or not ruta_variable.is_dir():
        raise FileNotFoundError(f"No existe la carpeta de la variable: {ruta_variable}")

    nc_files = sorted(ruta_variable.glob('*.nc'))
    if not nc_files:
        raise FileNotFoundError(f"No se encontraron archivos .nc en {ruta_variable}")

    modelos = [xr.open_dataset(str(archivo)) for archivo in nc_files]

    print(f"Se abrieron {len(modelos)} modelos para la variable '{variable}'.")
    return modelos

In [ ]:
modelos_clt = abrir_modelos('clt')
modelos_pr = abrir_modelos('pr')
modelos_rsds = abrir_modelos('rsds')
modelos_sfcWind = abrir_modelos('sfcWind')
modelos_tas = abrir_modelos('tas')

Como los modelos tienen diferentes tamamaños de grids, vemos el tamaño de los grids de cada modelo y lo convertimos todos en el mismo tamaño

In [ ]:
def info_grids(modelos):
    """
    Dada una lista de modelos (xarray.Dataset),
    devuelve un resumen con el tamaño y nombre de las coordenadas de cada grid.
    """
    info = []
    for i, ds in enumerate(modelos, start=1):
        # Buscar posibles nombres de coordenadas de lat/lon
        lat_name = next((c for c in ds.coords if 'lat' in c.lower()), None)
        lon_name = next((c for c in ds.coords if 'lon' in c.lower()), None)
        
        if lat_name and lon_name:
            lat_size = ds[lat_name].size
            lon_size = ds[lon_name].size
            info.append({
                'modelo': i,
                'lat_name': lat_name,
                'lon_name': lon_name,
                'lat_size': lat_size,
                'lon_size': lon_size
            })
        else:
            info.append({
                'modelo': i,
                'lat_name': lat_name,
                'lon_name': lon_name,
                'error': 'No se encontraron coordenadas lat/lon'
            })
    
    return info


Con mirar la lista para uno es suficiente ya que al usar los mismos modelos, en todos tendrían que tener el mismo tamaño

In [ ]:
grids_info = info_grids(modelos_clt)
for g in grids_info:
    print(g)

In [ ]:
def regrid_all(modelos, var_name, ref_index):
    """
    Regridea todas las variables 'var_name' de los modelos
    al grid del modelo de referencia indicado por ref_index.

    Args:
        modelos : list[xarray.Dataset | xarray.DataArray]
            Lista de modelos.
        var_name : str
            Nombre de la variable a regridear
        ref_index : int
            Índice del modelo de referencia en la lista
    
    Returns:
        list[xarray.DataArray]
    """
    regridded = []
    ref_model = modelos[ref_index]
    ref_grid = ref_model[var_name] if hasattr(ref_model, 'data_vars') else ref_model

    for i, model in enumerate(modelos):
        print(f"Procesando modelo {i} de {var_name}")
        data = model[var_name] if hasattr(model, 'data_vars') else model
        
        if i == ref_index:
            regridded.append(data)
        else:
            regridder = xe.Regridder(
                data,
                ref_grid,
                method='bilinear',
                extrap_method='nearest_s2d',
                reuse_weights=False
            )
            regridded.append(regridder(data))
    
    return regridded

In [ ]:
clt_regridded = regrid_all(modelos_clt, 'clt', 3)
pr_regridded = regrid_all(modelos_pr, 'pr', 3)
rsds_regridded = regrid_all(modelos_rsds, 'rsds', 3)
sfcWind_regridded = regrid_all(modelos_sfcWind, 'sfcWind', 3)
tas_regridded = regrid_all(modelos_tas, 'tas', 3)


Comprobar que efectivamente lo hicimos bien y tienen todos tamaño: 192x288

In [ ]:
primeros = [
    clt_regridded[0],
    pr_regridded[0],
    rsds_regridded[0],
    sfcWind_regridded[0],
    tas_regridded[0]
]
grids_info = info_grids(primeros)
for g in grids_info:
    print(g)


Calculo de medias mensuales

In [ ]:
def calc_monthly_means(modelos, var_name=None, start='1950-01-01', end='1980-12-31'):
    """
    Calcula la media mensual para una lista de modelos en el intervalo especificado.

    Args
        modelos : list[xarray.Dataset | xarray.DataArray]
            Lista de modelos o variables.
        var_name : str | None
            Nombre de la variable si los elementos son Datasets.
            Si ya son DataArray, déjalo en None.
        start, end : str
            Fechas para el rango temporal (slice).

    Returns:
        list[xarray.DataArray]
            Lista con las medias mensuales de cada modelo.
    """
    monthly_means = []
    
    for i, model in enumerate(modelos):

        # Extrae la variable si es Dataset
        data = model[var_name] if (var_name and hasattr(model, 'data_vars')) else model
        
        # Selección temporal y promedio mensual
        mean_month = (
            data
            .sel(time=slice(start, end))
            .groupby('time.month')
            .mean('time')
        )
        
        monthly_means.append(mean_month)
    
    return monthly_means


In [ ]:
clt_monthly_means = calc_monthly_means(clt_regridded, start='1950-01-01', end='1980-12-31')
pr_monthly_means = calc_monthly_means(pr_regridded, start='1950-01-01', end='1980-12-31')
rsds_monthly_means = calc_monthly_means(rsds_regridded, start='1950-01-01', end='1980-12-31')
sfcWind_monthly_means = calc_monthly_means(sfcWind_regridded, start='1950-01-01', end='1980-12-31')
tas_monthly_means = calc_monthly_means(tas_regridded, start='1950-01-01', end='1980-12-31')


Ahora calculamos la medía del ensemble, es decir, entre todos los modelos que tenemos para cada variable sacamos la medía

In [ ]:
def ensemble_mean(models_monthly):
    """
    Calcula la media del ensemble (promedio entre modelos)
    
    Parámetros
    ----------
    models_monthly : list[xarray.DataArray]
        Lista de DataArrays con las medias mensuales de cada modelo.
    
    Retorna
    -------
    xarray.DataArray
        DataArray con el promedio del ensemble.
    """
    combined = xr.concat(models_monthly, dim='model')
    return combined.mean(dim='model')

In [ ]:
clt_ensemble_mean = ensemble_mean(clt_monthly_means)
pr_ensemble_mean = ensemble_mean(pr_monthly_means)
rsds_ensemble_mean = ensemble_mean(rsds_monthly_means)
sfcWind_ensemble_mean = ensemble_mean(sfcWind_monthly_means)
tas_ensemble_mean = ensemble_mean(tas_monthly_means)

Exportar los datos a un .csv


In [ ]:
# Creamos las combinaciones lat-lon-time a partir de uno de los DataArray
lat = clt_ensemble_mean['lat'].values
lon = clt_ensemble_mean['lon'].values
time = clt_ensemble_mean['month'].values

# Crear el índice combinando todas las coordenadas
index = [f"{t}_{la}_{lo}" for t, la, lo in itertools.product(time, lat, lon)]

In [ ]:
df = pd.DataFrame({
    'clt_ensemble_mean': clt_ensemble_mean.values.flatten(),
    'pr_ensemble_mean': pr_ensemble_mean.values.flatten(),
    'rsds_ensemble_mean': rsds_ensemble_mean.values.flatten(),
    'sfcWind_ensemble_mean': sfcWind_ensemble_mean.values.flatten(),
    'tas_ensemble_mean': tas_ensemble_mean.values.flatten()
}, index=index)

In [ ]:
df.to_csv('ensemble_modelos.csv', index=True)

Exportar los datos a un .nc:

In [ ]:
# Asignar nombres
clt_ensemble_mean.name = 'clt_ensemble_mean'
pr_ensemble_mean.name = 'pr_ensemble_mean'
rsds_ensemble_mean.name = 'rsds_ensemble_mean'
sfcWind_ensemble_mean.name = 'sfcWind_ensemble_mean'
tas_ensemble_mean.name = 'tas_ensemble_mean'

# Combinar todos los DataArray en un Dataset
ds = xr.merge([
    clt_ensemble_mean,
    pr_ensemble_mean,
    rsds_ensemble_mean,
    sfcWind_ensemble_mean,
    tas_ensemble_mean
], compat='override')

# Añadir metadatos generales
ds.attrs = {
    'title': 'Climate Variable Ensemble Means',
    'description': 'Ensemble means calculated from monthly means for CLT, PR, RSDS, sfcWind, and TAS.',
    'author': '123',
    'Conventions': 'CF-1.8'
}

# Guardar a archivo NetCDF con compresión
encoding = {var: {"zlib": True, "complevel": 4} for var in ds.data_vars}
ds.to_netcdf("ensemble_modelos.nc", encoding=encoding)


Añadir los datos a un archivo .nc donde solo almacenamos las zonas terrestres y cortamos en los extremos (para nuestro objetivo hemos comprobado que las latitudes extremas generan datos demasiado desbalanceados que no nos interesan):

In [ ]:
lat = ds['lat'].values
lon = ds['lon'].values

# Mantener lon 0–360 y ordenarlas de oeste→este
lon_fixed = np.mod(lon, 360)
sort_idx = np.argsort(lon_fixed)
lon_fixed = lon_fixed[sort_idx]

# Reordenar dataset según longitudes corregidas
ds = ds.isel(lon=sort_idx)
ds = ds.assign_coords(lon=lon_fixed)

# Asegurar que las latitudes van de sur→norte
if np.any(np.diff(ds.lat.values) < 0):
    ds = ds.reindex(lat=list(reversed(ds.lat)))

# Asegurar orden correcto de dimensiones
for var in ds.data_vars:
    dims = ds[var].dims
    if 'lat' in dims and 'lon' in dims:
        other_dims = [d for d in dims if d not in ('lat', 'lon')]
        ds[var] = ds[var].transpose('lat', 'lon', *other_dims)


# Creamos máscara de tierra 
land_geom = cfeature.NaturalEarthFeature('physical', 'land', '110m').geometries() #con nuestros datos 110m es suficiente resolución
land_union = prep(unary_union(list(land_geom)))

# meshgrid con orden (lat, lon)
lat_grid, lon_grid = np.meshgrid(ds.lat.values, ds.lon.values, indexing="ij")
mask = np.zeros((len(ds.lat), len(ds.lon)), dtype=bool)

# Usar lon -180–180 solo internamente para comprobar tierra
for i in range(len(ds.lat)):
    for j in range(len(ds.lon)):
        lon_check = lon_grid[i, j] if lon_grid[i, j] <= 180 else lon_grid[i, j] - 360
        mask[i, j] = land_union.contains(Point(lon_check, lat_grid[i, j]))

mask_da = xr.DataArray(mask, coords=[ds.lat, ds.lon], dims=["lat", "lon"])


# Creamos máscara de latitudes extremas (-60 < lat > 45)
arctic_mask = (ds.lat < -60) | (ds.lat > 45)
arctic_mask_2d = arctic_mask.broadcast_like(mask_da)

# Aplicamos las máscaras
combined_mask = mask_da & ~arctic_mask_2d
ds_land = ds.where(combined_mask)  # Océanos y latitudes extremas → NaN

#  Añadir atributos CF correctos
ds_land['lon'].attrs = {
    'standard_name': 'longitude',
    'long_name': 'longitude',
    'units': 'degrees_east',
    'axis': 'X'
}
ds_land['lat'].attrs = {
    'standard_name': 'latitude',
    'long_name': 'latitude',
    'units': 'degrees_north',
    'axis': 'Y'
}

#  Metadatos globales
ds_land.attrs = ds.attrs
ds_land.attrs["description"] = (
    ds.attrs.get("description", "")
    + " (solo regiones terrestres excluyendo Ártico >60°N, orientado correctamente, lon 0–360)"
)
ds_land.attrs["note"] = (
    "Cumple CF-1.8: latitudes de sur→norte, longitudes de 0–360 de oeste→este, "
    "coordenadas con atributos estándar. Excluye océanos y regiones árticas (lat > 60°N)."
)

# Guardar a NetCDF comprimido
encoding = {var: {"zlib": True, "complevel": 4} for var in ds_land.data_vars}
output_name = "ensemble_modelos_land_temp.nc"
ds_land.to_netcdf(output_name, encoding=encoding)

print(f"Archivo guardado correctamente: {output_name}")